## XML output Parser

In [8]:
from langchain_openai import AzureChatOpenAI
from langchain_core.output_parsers import CommaSeparatedListOutputParser
from langchain_core.prompts import PromptTemplate
from langchain.output_parsers import XMLOutputParser

In [4]:
llm =AzureChatOpenAI(
  temperature=0,
  api_key="39f2089f6a2a4c52967260b0e9ffdc33",
  api_version="2024-02-01",
  azure_endpoint="https://dono-rag-demo-resource-instance.openai.azure.com",
  model="GPT_35_TURBO_DEMO_RAG_DEPLOYMENT_DONO"
)

In [5]:
model = llm

In [16]:
author_query = "Write all the books written by JK Rowling."
output = model.invoke(
    f"""{author_query}
Please enclose the books in <book></book> tags"""
)
print(output.content)

<book>Harry Potter and the Sorcerer's Stone</book>
<book>Harry Potter and the Chamber of Secrets</book>
<book>Harry Potter and the Prisoner of Azkaban</book>
<book>Harry Potter and the Goblet of Fire</book>
<book>Harry Potter and the Order of the Phoenix</book>
<book>Harry Potter and the Half-Blood Prince</book>
<book>Harry Potter and the Deathly Hallows</book>
<book>The Casual Vacancy</book>
<book>The Cuckoo's Calling (as Robert Galbraith)</book>
<book>The Silkworm (as Robert Galbraith)</book>
<book>Career of Evil (as Robert Galbraith)</book>
<book>Lethal White (as Robert Galbraith)</book>
<book>Troubled Blood (as Robert Galbraith)</book>


In [17]:
parser = XMLOutputParser()

prompt = PromptTemplate(
    template="""{query}\n{format_instructions}""",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | model | parser

output = chain.invoke({"query": author_query})
print(output)

{'books': [{'book': "Harry Potter and the Philosopher's Stone"}, {'book': 'Harry Potter and the Chamber of Secrets'}, {'book': 'Harry Potter and the Prisoner of Azkaban'}, {'book': 'Harry Potter and the Goblet of Fire'}, {'book': 'Harry Potter and the Order of the Phoenix'}, {'book': 'Harry Potter and the Half-Blood Prince'}, {'book': 'Harry Potter and the Deathly Hallows'}, {'book': 'The Casual Vacancy'}, {'book': "The Cuckoo's Calling"}, {'book': 'The Silkworm'}, {'book': 'Career of Evil'}, {'book': 'Lethal White'}, {'book': 'Troubled Blood'}]}


## YAML Output Parser 

In [10]:
from typing import List

from langchain.output_parsers import YamlOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI

In [11]:
class Joke(BaseModel):
    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")

In [12]:
joke_query = "Tell me a joke."

# Set up a parser + inject instructions into the prompt template.
parser = YamlOutputParser(pydantic_object=Joke)

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | model | parser

chain.invoke({"query": joke_query})

Joke(setup="Why don't skeletons fight each other?", punchline="They don't have the guts.")